<a href="https://colab.research.google.com/github/benjawad/AI-for-Environmental-Regulation-Analysis/blob/main/webscrapping_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# requirements

In [ ]:
!pip install selenium webdriver-manager beautifulsoup4 requests pymupdf
# Mise à jour critique des dépendances
!pip install -U bitsandbytes==0.43.1
!pip install -q langchain==0.2.1 langchain-community==0.2.1 faiss-cpu==1.8.0 sentence-transformers==3.0.0 pypdf==4.2.0 accelerate==0.30.1 transformers==4.41.2
!pip install pdfplumber transformers accelerate langchain huggingface_hub --upgrade
!pip install selenium webdriver-manager beautifulsoup4 requests pymupdf  playwright  chromium
!pip install -U langchain langchain-community huggingface_hub pdfplumber transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 65.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!playwright install > /dev/null 2>&1

# importante libaries

# scrapping

In [ ]:
import asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from playwright.async_api import async_playwright

BASE_URL = "https://www.sgg.gov.ma/BulletinOfficiel.aspx"
pdf_links = set()
MAX_PAGES = 4

async def extract_pdfs_from_frame(frame):
    content = await frame.content()
    soup = BeautifulSoup(content, "html.parser")
    new_links = set()

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.lower().endswith(".pdf"):
            full_link = urljoin(BASE_URL, href)
            new_links.add(full_link)

    return new_links

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(BASE_URL)
        await page.wait_for_load_state("networkidle")

        # Find and switch to content iframe
        frame = None
        for f in page.frames:
            if "BulletinOfficiel" in f.url:
                frame = f
                break

        if not frame:
            print("⚠️ Content frame not found! Using main frame")
            frame = page.main_frame

        current_page = 1
        previous_links = set()

        while current_page <= MAX_PAGES:
            print(f"📄 Scraping page {current_page}...")

            # Extract PDF links
            new_links = await extract_pdfs_from_frame(frame)
            new_count = len(new_links - pdf_links)
            pdf_links.update(new_links)
            print(f"   ➕ Found {new_count} new PDFs (Total: {len(pdf_links)})")

            # Check for duplicate content (end of pages)
            if new_links and new_links == previous_links:
                print("🛑 Duplicate content detected - stopping pagination")
                break
            previous_links = new_links

            # Find next button by text
            next_buttons = await frame.query_selector_all("a")
            next_button = None

            for btn in next_buttons:
                text = await btn.text_content()
                if text and "Suivant" in text.strip():
                    next_button = btn
                    break

            if not next_button:
                print("⏹️ Next button not found - stopping pagination")
                break

            # Click next button and wait
            await next_button.click()
            await asyncio.sleep(2)  # Allow time for content load
            current_page += 1

        await browser.close()
        print(f"\n✅ Scraping completed. Total unique PDFs: {len(pdf_links)}")
        for link in sorted(pdf_links):
            print(link)

await run()


📄 Scraping page 1...
   ➕ Found 20 new PDFs (Total: 20)
📄 Scraping page 2...
   ➕ Found 10 new PDFs (Total: 30)
📄 Scraping page 3...
   ➕ Found 10 new PDFs (Total: 40)
📄 Scraping page 4...
   ➕ Found 10 new PDFs (Total: 50)

✅ Scraping completed. Total unique PDFs: 50
https://www.sgg.gov.ma/BO/FR/2873/2022/ECI_11_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2022/ECI_12_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2022/ECI_13_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2022/ECI_14_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2023/ECI_15_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2023/ECI_16_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2023/ECI_17_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2023/ECI_18_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2024/BO_7274_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2024/BO_7280_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2024/BO_7284_Fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2024/BO_7288_fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/2024/BO_7292_fr.pdf
https://www.sgg.gov.ma/BO/FR/2873/202

In [ ]:
import os
import fitz
import requests

os.makedirs("/pdfs", exist_ok=True)
pdf_texts = []
for url in pdf_links:
    filename = os.path.basename(url)
    path = os.path.join("/pdfs", filename)

    try:
        r = requests.get(url)
        with open(path, "wb") as f:
            f.write(r.content)

        doc = fitz.open(path)
        text = ""
        for page in doc:
            text += page.get_text()
        pdf_texts.append((filename, text[:1000]))
    except Exception as e:
        print(f"❌ Failed on {filename}: {e}")

# Preview sample content
pdf_names = []
for name, content in pdf_texts[:2]:
    print(f"\n📄 {name}\n{'-'*40}\n{content}")




📄 BO_7336_fr.pdf
----------------------------------------
Pages
Pages
{
ROYAUME DU MAROC
BULLETIN OFFICIEL
EDITION DE TRADUCTION OFFICIELLE
EDITIONS
TARIFS D’ABONNEMENT
ABONNEMENT
IMPRIMERIE OFFICIELLE
Rabat - Chellah
Tél. : 05.37.76.50.24 ‑ 05.37.76.50.25
05.37.76.54.13
Compte n° :
310 810 1014029004423101 33
ouvert à la Trésorerie Préfectorale de Rabat 
au nom du régisseur des recettes 
de l’Imprimerie officielle
AU MAROC
A L’ETRANGER
6 mois
1 an
Edition générale...................................................................
Edition de traduction officielle.............................................
Edition des conventions internationales................................
Edition des annonces légales, judiciaires et administratives...
Edition des annonces relatives à l’immatriculation foncière..
250 DH
150 DH
150 DH
250 DH
250 DH
400 DH
200 DH
200 DH
300 DH
300 DH
A destination de l’étranger, 
par voies ordinaire, aérienne 
ou de la poste rapide interna‑
tionale, les tarifs prév

In [ ]:
pdf_names = []
for name, content in pdf_texts:
    pdf_names.append("/pdfs/"+name)


In [ ]:
len(pdf_names)
pdf_names

['/pdfs/BO_7336_fr.pdf',
 '/pdfs/BO_7305-bis_fr.pdf',
 '/pdfs/BO_7332_fr.pdf',
 '/pdfs/BO_7350_fr.pdf',
 '/pdfs/BO_7296_fr.pdf',
 '/pdfs/ECI_16_Fr.pdf',
 '/pdfs/ECI_12_Fr.pdf',
 '/pdfs/BO_7358_fr.pdf',
 '/pdfs/BO_7314_Fr.pdf',
 '/pdfs/BO_7404_fr.pdf',
 '/pdfs/BO_7418_fr.pdf',
 '/pdfs/BO_7388_Fr.pdf',
 '/pdfs/BO_7355-bis_fr.pdf',
 '/pdfs/BO_7366_fr.pdf',
 '/pdfs/ECI_15_Fr.pdf',
 '/pdfs/BO_7354_fr.pdf',
 '/pdfs/BO_7380_fr.pdf',
 '/pdfs/BO_7407-bis_Fr.pdf',
 '/pdfs/ECI_13_Fr.pdf',
 '/pdfs/ECI_19_Fr.pdf',
 '/pdfs/ECI_20_Fr.pdf',
 '/pdfs/BO_7376_fr.pdf',
 '/pdfs/BO_7322_fr.pdf',
 '/pdfs/BO_7410_fr.pdf',
 '/pdfs/BO_7384_fr.pdf',
 '/pdfs/ECI_14_Fr.pdf',
 '/pdfs/BO_7340_fr.pdf',
 '/pdfs/BO_7342-bis_fr.pdf',
 '/pdfs/BO_7284_Fr.pdf',
 '/pdfs/BO_7344_fr.pdf',
 '/pdfs/ECI_11_Fr.pdf',
 '/pdfs/BO_7348-bis_fr.pdf',
 '/pdfs/BO_7414_fr.pdf',
 '/pdfs/BO_7370_fr.pdf',
 '/pdfs/BO_7318_fr.pdf',
 '/pdfs/ECI_18_Fr.pdf',
 '/pdfs/BO_7288_fr.pdf',
 '/pdfs/BO_7326_fr.pdf',
 '/pdfs/BO_7396_Fr.pdf',
 '/pdfs/BO_730

# Model data extraction

In [ ]:
import re
import torch
import json
import fitz  # PyMuPDF
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoTokenizer, T5ForConditionalGeneration

# --- Model Initialization ---
device = 0 if torch.cuda.is_available() else -1
device_name = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device_name}")

# Initialize models
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": device_name}
)

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
)

# --- Default Data and Rules ---
DEFAULT_PROJECT_TEXT = """
The project execution plans translate our contractual and legislative requirements, Customer expectations,
and our business processes into the way we intend to execute the Chemical Additive Plant. This Project Execution Plan describes:
Specific actions required to set up the systems and processes for delivering the project
Any deviations from JESA's core business processes, as defined in the Management System, required for this project.
This document aligns with Life - our safety and wellbeing approach.

Project Outline:
The Customer is NOVADDIX, subsidiary of INNOVX, a new entity dedicated to product and commercialize Chemical additives.
The project is based on a technology developed by UMP6P (Mr BENHIDA & Mr MAKAN) and the process has been tested in a pilot plant at Jorf Lasfar with a capacity of 5T/Week.
The project will be developed to construct a chemical additives plant able to produce a capacity of 20000 T/Y and 75 T/D with the following aspect:
Domain: Industrial - Chemical.
Product Class: Flammable and Toxic Products.
"""

# --- Inference Rules ---
inference_rules = {
    "sector": {
        "patterns": ["sector", "domain"],
        "extract": r"Domain:\s*(.*?)[.\n]",
        "clean": lambda x: x.strip()
    },
    "customer": {
        "patterns": ["customer", "client", "sponsor"],
        "extract": r"The Customer is\s*(.*?)[,.\n]",
        "clean": lambda x: x.strip()
    },
    "technology": {
        "patterns": ["technology"],
        "extract": r"technology developed by\s*(.*?)\s*(and|\.|,|\n)",
        "clean": lambda x: re.sub(r"\([^)]*\)", "", x).strip()
    },
    "duration": {
        "patterns": ["duration", "timeline"],
        "infer": lambda text: "2-3 years (estimated from capacity scaling 5T/week → 75T/day)"
        if "pilot" in text.lower() else "Duration not specified"
    },
    "safety": {
        "patterns": ["safety", "concern"],
        "infer": lambda text: "High risk: Requires special handling for flammable and toxic materials"
        if ("flammable" in text.lower() and "toxic" in text.lower())
        else "Safety concerns not specified"
    },
    "location": {
        "patterns": ["location", "pilot plant"],
        "extract": r"pilot plant at\s*(.*?)\s*(with|\.|,|\n)",
        "clean": lambda x: x.strip()
    },
    "capacity": {
        "patterns": ["production capacity", "capacity"],
        "extract": r"produce a capacity of\s*(.*?)\s*(with|\.|,|\n)",
        "clean": lambda x: x.strip()
    }
}

# --- Core Functions ---
def build_vector_store(texts):
    """Builds a FAISS vector store from texts"""
    if not texts:
        print("No texts provided to build vector store.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=768,  # Réduit pour éviter les erreurs de token
        chunk_overlap=96,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    chunks = text_splitter.create_documents(texts)
    print(f"Created {len(chunks)} chunks for the vector store.")
    return FAISS.from_documents(chunks, embedding_model)

def clean_rag_answer(answer, question):
    """Nettoie les réponses RAG en fonction du type de question"""
    # Détection des questions sur les seuils d'émission
    if "seuil d'émission" in question.lower():
        # Regex pour extraire spécifiquement les valeurs de seuils
        pattern = r"(\d+[\s\u00A0]*[\u00B5mg]?g?/?m³?|mg/Nm³)"
        matches = re.findall(pattern, answer)
        if matches:
            return " | ".join(set(matches))  # Valeurs uniques

        # Vérification des seuils d'information vs émission
        if "μg/m³" in answer or "µg/m³" in answer:
            return "[ERREUR] Ceci semble être un seuil de qualité d'air, non d'émission industriel"

    # Correction spécifique pour CO vs CO₂
    if "CO₂" in question and "monoxyde" in answer:
        answer = answer.replace("monoxyde", "dioxyde").replace("CO", "CO₂")

    # Réduction des réponses trop longues
    if len(answer) > 300:
        sentences = answer.split('.')
        key_sentences = [s for s in sentences if any(kw in s for kw in ["limite", "seuil", "valeur", "autorisation"])]
        return '. '.join(key_sentences[:3]) + '.' if key_sentences else '. '.join(sentences[:3]) + '...'

    return answer

def rag_query(question, vector_store, k=8):
    """
    Performs contextual search using RAG model with improved prompts
    """
    # Détection des sujets non couverts
    UNCOVERED_TOPICS = {
        "rejets liquides": "documents couvrent uniquement les émissions atmosphériques",
        "eaux usées": "documents couvrent uniquement les émissions atmosphériques",
        "bruit": "aucune donnée dans les documents fournis",
        "zones protégées": "information non disponible dans les documents"
    }

    for topic, msg in UNCOVERED_TOPICS.items():
        if topic in question.lower():
            return f"Aucune information pertinente ({msg})"

    if not vector_store:
        return "Vector store not available."

    docs = vector_store.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Prompt amélioré avec instructions strictes
    prompt = f"""
    En te basant STRICTEMENT sur le contexte ci-dessous :
    1. Réponds EXCLUSIVEMENT à la question posée
    2. Pour les valeurs numériques : donne UNIQUEMENT la valeur et l'unité
    3. Si la question concerne les ÉMISSIONS INDUSTRIELLES, ignore les seuils de qualité d'air
    4. Si l'information est absente : "Aucune information pertinente"

    Contexte:
    {context}

    Question: {question}
    Réponse concise:
    """

    raw_answer = generator(prompt, max_length=256)[0]['generated_text'].strip()

    # Appliquer le nettoyage
    return clean_rag_answer(raw_answer, question)

def extract_text_from_pdf_file(file_path):
    """Extracts text from PDF with error handling and natural ordering"""
    try:
        doc = fitz.open(file_path)
        text = []
        for page in doc:
            text.append(page.get_text("text", sort=True))  # Tri pour ordre naturel
        doc.close()
        return "\n".join(text)
    except Exception as e:
        print(f"Error reading PDF {file_path}: {e}")
        return ""

def get_enhanced_answer(question, context):
    """
    Gets answer with improved fallback mechanism
    """
    # Rule-based extraction first
    for key, rule in inference_rules.items():
        if any(pattern in question.lower() for pattern in rule["patterns"]):
            if "extract" in rule:
                match = re.search(rule["extract"], context, re.IGNORECASE)
                if match:
                    answer = match.group(1)
                    return rule.get("clean", lambda x: x)(answer)
            if "infer" in rule:
                return rule["infer"](context)

    # Fallback to generator with strict instructions
    prompt = f"""
    Réponds à la question en utilisant uniquement le contexte fourni.
    Si l'information n'est pas dans le contexte, réponds "Information non trouvée".
    Important :donner moi les sources .
    Contexte:
    {context}

    Question: {question}
    Réponse:
    """

    return generator(prompt, max_length=256)[0]['generated_text'].strip()

def analyze_project_local_multiple_pdfs(project_text=DEFAULT_PROJECT_TEXT, pdf_paths=[]):
    """
    Main analysis function with enhanced processing
    """
    results = {"Project Text": {}, "Regulatory": {}}

    # 1. Process project description
    print("--- Analyzing Project Text ---")
    project_questions = [
        "What is the sector of the project?",
        "Where is the pilot plant located?",
        "What technology is the project based on?",
        "What is the estimated duration of the project?",
        "Who is the Customer?",
        "What are the safety concerns?",
        "What is the production capacity?" ,
        "What is the main purpose of the Commitment Register?",
        "What is the definition of a commitment and how is it managed?",
        "What are the responsibilities when initiating and customizing the Commitment Register?",
        "What is the role of the Project Manager in maintaining the register?",
        "How is the technical review of the Commitment Register performed?",
        "Who are the members of the Sustainability Steering Committee?",
        "What are the references mentioned in the Commitment Register document?",
        "What are the abbreviations and their meanings listed in the Commitment Register?"



    ]

    for question in project_questions:
        answer = get_enhanced_answer(question, project_text)
        results["Project Text"][question] = answer
        print(f"Q: {question}\nA: {answer}\n")

    # 2. Process regulatory documents
    print("--- Analyzing Regulatory PDFs ---")
    regulatory_texts = []
    valid_pdf_paths = []

    for pdf_path in pdf_paths:
        print(f"Processing: {pdf_path}")
        text = extract_text_from_pdf_file(pdf_path)
        if text:  # Skip empty texts
            regulatory_texts.append(text)
            valid_pdf_paths.append(pdf_path)

    vector_store = build_vector_store(regulatory_texts) if regulatory_texts else None

    # 3. Regulatory questions with domain-aware filtering
    law_questions = [
        "Quelles sont les autorisations nécessaires pour ce type de projet ?",
        "Quelles sont les règles environnementales à respecter ?",
        "Quelles sont les contraintes légales spécifiques au secteur ?",
        "Quels documents doivent être fournis pour la conformité réglementaire ?",
        # "Quel est le seuil d'émission autorisé pour le CO₂ dans ce type de projet ?",
        # "Quel est le seuil d'émission autorisé pour le SO₂ dans ce type d'installation ?",
        # "Quels sont les niveaux maximums admissibles pour les rejets liquides ?",
        # "Existe-t-il des restrictions concernant le rejet des eaux usées ?",
        # "Y a-t-il des exigences sur la gestion des déchets industriels dangereux ?",
        # "Quels équipements de contrôle de pollution doivent être installés ?",
        # "Ce projet est-il soumis à une étude d’impact environnemental obligatoire ?",
        # "Quels sont les seuils de bruit autorisés pour les installations industrielles ?",
        # "Y a-t-il des zones protégées ou sensibles à proximité du site ?",
        # "Quels sont les protocoles de sécurité obligatoires pour les produits toxiques et inflammables ?",
        # "Quelles obligations de suivi et de reporting environnemental sont requises ?",
        # "Existe-t-il des lois spécifiques au Maroc concernant les produits chimiques industriels ?",
    ]

    print("\n--- Answering Regulatory Questions with RAG ---")
    for question in law_questions:
        if vector_store:
            answer = rag_query(question, vector_store, k=8)
        else:
            answer = "Aucun document réglementaire valide fourni"

        results["Regulatory"][question] = answer
        print(f"Q: {question}\nA: {answer}\n")

    # 4. Document processing summary
    results["Processed Documents"] = {
        "Project Text Processed": True,
        "Regulatory PDFs Succeeded": valid_pdf_paths,
        "Regulatory PDFs Failed": [pdf for pdf in pdf_paths if pdf not in valid_pdf_paths]
    }

    return results

# --- Main Execution ---
if __name__ == "__main__":
    pdf_files = pdf_names


    final_results = analyze_project_local_multiple_pdfs(pdf_paths=pdf_files)

    print("\n\n--- FINAL RESULTS ---")
    print(json.dumps(final_results, indent=2, ensure_ascii=False))

Using device: cuda


Device set to use cuda:0


--- Analyzing Project Text ---
Q: What is the sector of the project?
A: Industrial - Chemical

Q: Where is the pilot plant located?
A: Jorf Lasfar

Q: What technology is the project based on?
A: UMP6P

Q: What is the estimated duration of the project?
A: 2-3 years (estimated from capacity scaling 5T/week → 75T/day)

Q: Who is the Customer?
A: NOVADDIX

Q: What are the safety concerns?
A: High risk: Requires special handling for flammable and toxic materials

Q: What is the production capacity?
A: 20000 T/Y and 75 T/D

Q: What is the main purpose of the Commitment Register?
A: Information non trouvée

Q: What is the definition of a commitment and how is it managed?
A: Specific actions required to set up the systems and processes for delivering the project Any deviations from JESA's core business processes, as defined in the Management System, required for this project

Q: What are the responsibilities when initiating and customizing the Commitment Register?
A: Specific actions required 

Token indices sequence length is longer than the specified maximum sequence length for this model (703 > 512). Running this sequence through the model will result in indexing errors



--- Answering Regulatory Questions with RAG ---
Q: Quelles sont les autorisations nécessaires pour ce type de projet ?
A:  No 7404 – 17 kaada 1446 (15-5-2025) BULLETIN OFFICIEL 615 ou peut doit inclu pouvant proposé collations l’établissement » Autorisations d’acquisition et construction d’immeubles : sur demande de certains mutuelles, l’Autorité a accordé les autorisations suivantes : .  Une autorisation pour la construction du siège d’une uvre sociale ; .  Une autorisation pour la reconstruction d’un local pour une délégation régionale ; .



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Q: Quelles sont les règles environnementales à respecter ?
A: Article 13 : Règles afférentes aux émissions de santé L’Operateur s’engage à respecter les dispositions légales et règlementaires en vigueur, notamment les décisions et recommandations du Conseil Supérieur relatives aux programmes traitant de la santé

Q: Quelles sont les contraintes légales spécifiques au secteur ?
A:  législationles par nombre la d'une prévue, des et en d'entretien de en et exploiterpar ou de l'autorisation de l'exploitation du terrain (Art.

Q: Quels documents doivent être fournis pour la conformité réglementaire ?
A: à et sur doivent de de être équipements réglementation à et sur doivent de de be équipements réglementation dispositions norme et respecter transats, doivent Dans doivent doit à en le l'établissement l'établissement réglementation être les en marche.  législationles les cette la de d'une prévue, de d'entretien l'établissement l'établissement réglementation être de marche...



--- FINAL RESU